In [1]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, MetaData, text
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
load_dotenv()
DB_URL = os.getenv("DB_VM_URL")
engine = create_engine(DB_URL)
conn = engine.connect()
metadata = MetaData()
metadata.reflect(bind=engine)

In [17]:
# select 26 rows from SQL table to insert in dataframe.
query = text('''
            select c.ContactPersoonId, i.CampagneId, i.CampagneNaam, ca.Startdatum, a.Ondernemingsaard, a.Ondernemingstype, a.PrimaireActiviteit, f.Naam as Functie
            from Contactfiche c
            join Account a on a.AccountId = c.AccountId
            join Inschrijving i on i.ContactficheId = c.ContactPersoonId
            join Campagne ca on ca.CampagneId = i.CampagneId
            join ContactficheFunctie cf on cf.ContactpersoonId = c.ContactPersoonId
            join Functie f on f.FunctieId = cf.FunctieId
            where i.CampagneId is not null and a.status = 'actief';
            ''')
result = conn.execute(query)

#convert to dataframe
df_omschrijving = pd.DataFrame(result.fetchall())
df_omschrijving.set_index('ContactPersoonId', inplace=True)
df_omschrijving["rating"] = 5

query2 = text('''
                with pageview_count as(
                    select p.PageTitle, count(p.ContactId) as count
                    from Pageviews p
                    group by p.PageTitle
                )
                select PageTitle, ContactId
                from Pageviews
                where Pagetitle in (select top (2500) PageTitle
                from pageview_count
                order by count desc)
              ''')

result2 = conn.execute(query2)
df_pageviews = pd.DataFrame(result2.fetchall())
df_pageviews.set_index('ContactId', inplace=True)
df_pageviews["rating"] = 1


df_omschrijving.head()

,CampagneId,CampagneNaam,Startdatum,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,,
D7DDF02E-9F72-E111-B43A-00505680000A,5CC8251F-104B-E911-80FC-001DD8B72B62,OV-NW-Voka Politica Gent Zuid 2019,2019-03-20 10:30:00,Productie,Bedrijf,"Chemie, petrochemie",Medewerker,5
85A2204B-733D-EC11-8125-001DD8B72B61,A8949A21-6CEE-ED11-8849-6045BD8956C9,OV-NW&IN-EDIH-DIGITALIS-XpertFinder-Data-AI,2023-10-19 08:00:00,Diensten,Bedrijf,Consultancy,Medewerker Commercieel,5
CEB69229-0CFB-E511-96DE-005056B06EB4,B4E3E30A-E6CA-EC11-A7B5-000D3A20A90F,OV-NW/DO-Quick Refresh-Webinar-Energiedelen,2022-06-15 11:00:00,Productie & Diensten,None,Bouw,Bedrijfsleider,5
F9877768-7826-EC11-8127-001DD8B72B62,FEEB8AD3-2FD4-EB11-8124-001DD8B72B62,OV-IN-Lab: Change Management in digitalisering,2022-05-10 09:00:00,Diensten,Bedrijf,Bouw,Verantwoordelijke IT & Digitalisering,5
0164420E-9596-EA11-8111-001DD8B72B62,74313A3B-E88D-EA11-810F-001DD8B72B61,OV-Webinar: E-commerce op de Chinese markt,2020-05-20 15:30:00,None,Bedrijf,None,Medewerker Commercieel,5


In [18]:
df_pageviews.head()

,PageTitle,rating
ContactId,,
71C7CDFA-379C-EB11-811E-001DD8B72B62,Weer hogere loonindexering op komst in 2024 | ...,1
2A04879B-1141-EC11-8125-001DD8B72B61,Nieuwjaarsinvitito 2023 | Voka,1
1E9BCBC0-9025-EC11-8124-001DD8B72B61,Netwerkevents | Voka,1
3F1EB182-8E3C-E811-80EF-001DD8B72B62,Marketing | Voka,1
5073196D-6611-E811-80EF-001DD8B72B61,Bedrijfsafval: dit verandert op 1 januari | Voka,1


In [19]:
df_pivot_omschrijving = pd.pivot_table(df_omschrijving, index='ContactPersoonId', columns=['Ondernemingsaard', 'Ondernemingstype', 'PrimaireActiviteit', 'Functie'], values='rating', fill_value = 0)
df_pivot_omschrijving

Ondernemingsaard                                      Diensten             \
Ondernemingstype                                       Bedrijf              
PrimaireActiviteit                   Accountancy & boekhouding              
Functie                                         Bedrijfsleider Bestuurder   
ContactPersoonId                                                            
00169619-E322-E911-80FB-001DD8B72B62                         0          0   
0017416A-2C6E-E111-B43A-00505680000A                         0          0   
0019C15A-6481-E611-80DE-001DD8B72B61                         0          0   
00223C8E-467F-E311-BBFD-005056B06EB4                         0          0   
00231824-53EA-ED11-8849-6045BD895420                         0          0   
...                                                        ...        ...   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                         0          0   
FFF68074-EB93-E911-80FF-001DD8B72B62                         0          0   
FFF68536-5DE0-E111-8A53-984BE17C2819                         0          0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                         0          0   
FFFEA9CB-ED93-EC11-B400-000D3A2B10EB                         0          0   

Ondernemingsaard                                           \
Ondernemingstype                                            
PrimaireActiviteit                                          
Functie                              Contact Lidmaatschap   
ContactPersoonId                                            
00169619-E322-E911-80FB-001DD8B72B62                    0   
0017416A-2C6E-E111-B43A-00505680000A                    0   
0019C15A-6481-E611-80DE-001DD8B72B61                    0   
00223C8E-467F-E311-BBFD-005056B06EB4                    0   
00231824-53EA-ED11-8849-6045BD895420                    0   
...                                                   ...   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                    0   
FFF68074-EB93-E911-80FF-001DD8B72B62                    0   
FFF68536-5DE0-E111-8A53-984BE17C2819                    0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                    0   
FFFEA9CB-ED93-EC11-B400-000D3A2B10EB                    0   

Ondernemingsaard                                           \
Ondernemingstype                                            
PrimaireActiviteit                                          
Functie                              Directie of kaderlid   
ContactPersoonId                                            
00169619-E322-E911-80FB-001DD8B72B62                    0   
0017416A-2C6E-E111-B43A-00505680000A                    0   
0019C15A-6481-E611-80DE-001DD8B72B61                    0   
00223C8E-467F-E311-BBFD-005056B06EB4                    0   
00231824-53EA-ED11-8849-6045BD895420                    0   
...                                                   ...   
FFF0C643-DAE4-ED11-A7C7-000D3A4AB78E                    0   
FFF68074-EB93-E911-80FF-001DD8B72B62                    0   
FFF68536-5DE0-E111-8A53-984BE17C2819                    0   
FFFAE2B6-11D5-EC11-A7B5-000D3ABD1F85                    0   
FFFEA9CB-ED93-EC11-B400-000D3A2B10EB                    0   

Ondernemingsaard                                                      \
Ondernemingstype                                                       
PrimaireActiviteit                                                     
Functie                              Management Assistent Medewerker   
ContactPersoonId                                                       
00169619-E322-E911-80FB-001DD8B72B62                    0          0   
0017416A-2C6E-E111-B43A-00505680000A                    0          0   
0019C15A-6481-E611-80DE-001DD8B72B61                    0          0   
00223C8E-467F-E311-BBFD-005056B06EB4                    0          0   
00231824-53EA-ED11-8849-6045BD895420                    0          0   
...                                                   ...        ...   
FFF0C643-DAE4-ED11-A7

In [20]:
df_pivot_pageviews = pd.pivot_table(df_pageviews, index='ContactId', columns=['PageTitle'], values='rating', fill_value = 0)
df_pivot_pageviews

PageTitle,"""Aansluiten bij een lerend netwerk van Voka is de investering meer dan waard!"" | Voka","""Al 100 hectare industriegebied aan Albertkanaal geschrapt"" | Voka","""Bij drie op tien bedrijven zijn loonkosten tot 10% gestegen"" | Voka","""Bijkomende droogtemaatregelen zijn nefast voor Limburgse ondernemingen"" | Voka","""Bouwen doe je met een blik op 10 jaar verder"" | Voka","""De Luchtige Kant"": de kalkoen van Suzy Cretskens | Voka","""Een authentieke cultuur start voor ons bij leiderschap"" | Voka","""FIRST Tech Challenge als middel tegen mismatch op de arbeidsmarkt? | Voka","""Het VCDO-traject van Voka geeft richting en structuur aan ons duurzaamheidsbeleid."" | Voka","""Keep calm and go international."" | Voka",...,hier voor de nieuwe folder van 2022.,inhoud 'Gun jezelf en je onderneming een BTW-update | Voka,jaarverslag 2022 | Voka,vokaadmin | Voka,vzw - interesse Bryo ScaleUp | Voka,vzw - interesse Bryo StandUp | Voka,vzw - interesse Bryo StartUp | Voka,vzw - interesse Mercurio Go International | Voka,vzw - interesse Tech@venture | Voka,| Voka
ContactId,,,,,,,,,,,,,,,,,,,,,
00013C8A-6F1C-E211-9DAA-005056B06EB4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00017E48-212C-EC11-8127-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002EDD7-2405-E511-ABE8-005056B06EB4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0006CEC0-931A-EB11-8117-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000C5DD4-016A-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FF76B927-E283-E611-80E3-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FF9A9C90-DC1A-E711-80E4-001DD8B72B62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FFD3EBB0-F16C-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#join both dataframes
#df_pivot = df_pivot_omschrijving.join(df_pivot_pageviews, how='inner')
df_pivot = pd.concat([df_pivot_omschrijving, df_pivot_pageviews], axis=1, join='inner')
df_pivot.head()

,"(Diensten, Bedrijf, Accountancy & boekhouding, Bedrijfsleider)","(Diensten, Bedrijf, Accountancy & boekhouding, Bestuurder)","(Diensten, Bedrijf, Accountancy & boekhouding, Contact Lidmaatschap)","(Diensten, Bedrijf, Accountancy & boekhouding, Directie of kaderlid)","(Diensten, Bedrijf, Accountancy & boekhouding, Management Assistent)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Commercieel)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Communicatie)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Financieel)","(Diensten, Bedrijf, Accountancy & boekhouding, Medewerker Juridische dienst)",...,hier voor de nieuwe folder van 2022.,inhoud 'Gun jezelf en je onderneming een BTW-update | Voka,jaarverslag 2022 | Voka,vokaadmin | Voka,vzw - interesse Bryo ScaleUp | Voka,vzw - interesse Bryo StandUp | Voka,vzw - interesse Bryo StartUp | Voka,vzw - interesse Mercurio Go International | Voka,vzw - interesse Tech@venture | Voka,| Voka
0017416A-2C6E-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00426A48-F851-EC11-8C62-000D3ABFC672,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
0069C9F7-B76A-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00B3AC4A-E76C-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00E23D3F-1B6F-E111-B43A-00505680000A,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
similarity_matrix = cosine_similarity(df_pivot, [df_pivot.loc["01B45481-0877-E911-80FE-001DD8B72B62"]]).reshape(1,-1)[0]
similarity_matrix

array([0.        , 0.04401188, 0.        , ..., 0.03925343, 0.        ,
       0.        ])

In [23]:
select_contact = "9F266F3D-0A62-EA11-810D-001DD8B72B61"

similarities = df_pivot.index.join(similarity_matrix)
similarities = pd.DataFrame({'ContactPersoonId':df_pivot.index, 'sim':similarity_matrix}).set_index('ContactPersoonId')

similar_users = similarities[similarities['sim'] > 0.5].sort_values(by='sim', ascending=False)
similar_users

,sim
ContactPersoonId,
01B45481-0877-E911-80FE-001DD8B72B62,1.000000
5AC0141B-D8ED-EB11-8124-001DD8B72B62,0.929735
7CBB14DB-4165-EB11-811B-001DD8B72B62,0.911505
41E272E4-CFC6-EC11-A7B6-000D3A497120,0.911505
0A7B8EE9-C36B-E111-B43A-00505680000A,0.911505
...,...
EAB7DEE4-59E6-E811-80F8-001DD8B72B61,0.594228
125150ED-82A3-EC11-983F-00224884C408,0.583713
303EBDC8-660C-EB11-8114-001DD8B72B61,0.555786


In [24]:
done_campaigns = df_omschrijving.loc[df_omschrijving.index == select_contact]['CampagneId']
done_campaigns

ContactPersoonId
9F266F3D-0A62-EA11-810D-001DD8B72B61    29668F0F-6C86-EB11-811A-001DD8B72B61
9F266F3D-0A62-EA11-810D-001DD8B72B61    B5DB7D00-F095-EA11-8111-001DD8B72B62
9F266F3D-0A62-EA11-810D-001DD8B72B61    A3E61415-BC1D-EB11-8115-001DD8B72B61
Name: CampagneId, dtype: object

In [25]:
similar_campagnes = df_omschrijving[df_omschrijving.index.isin(similar_users.index)].replace(0, np.nan).dropna(axis=1, how='all')
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,,
0A7B8EE9-C36B-E111-B43A-00505680000A,24F95D75-F0F7-EA11-8115-001DD8B72B62,OV-Stad Aalst- 4 webinars,2020-10-21 12:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
0A7B8EE9-C36B-E111-B43A-00505680000A,68BCCA0B-838E-E811-80F2-001DD8B72B62,OV-NW-Connect Aalst 2019,2019-01-17 19:30:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
0A7B8EE9-C36B-E111-B43A-00505680000A,DB733356-A23D-ED11-9DB0-6045BD8952CE,OV-NW-Nieuwjaarsreceptie Aalst 2023,2023-01-26 18:30:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
0A7B8EE9-C36B-E111-B43A-00505680000A,DB733356-A23D-ED11-9DB0-6045BD8952CE,OV-NW-Nieuwjaarsreceptie Aalst 2023,2023-01-26 18:30:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
0A7B8EE9-C36B-E111-B43A-00505680000A,24F95D75-F0F7-EA11-8115-001DD8B72B62,OV-Stad Aalst- 4 webinars,2020-10-21 12:00:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
...,...,...,...,...,...,...,...,...
3A58CA88-5FFF-EB11-8125-001DD8B72B62,CD9F2FDB-3FB8-ED11-83FF-6045BD895D85,OV-NW- Voka Visit Graanstokerij Braeckman,2023-04-20 18:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
3A58CA88-5FFF-EB11-8125-001DD8B72B62,BE613E71-6C5F-ED11-9561-6045BD895D85,OV-NW-Voka Politica Oudenaarde 2022,2022-12-09 08:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
3A58CA88-5FFF-EB11-8125-001DD8B72B62,F98F51AE-0C41-EB11-8118-001DD8B72B62,OV-IN-Innovatieforum 2022a,2022-02-01 13:30:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5


In [26]:
#remove double campagneIds in similar_campagnes
similar_campagnes = similar_campagnes.drop_duplicates(subset=['CampagneId'])
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,,
0A7B8EE9-C36B-E111-B43A-00505680000A,24F95D75-F0F7-EA11-8115-001DD8B72B62,OV-Stad Aalst- 4 webinars,2020-10-21 12:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
0A7B8EE9-C36B-E111-B43A-00505680000A,68BCCA0B-838E-E811-80F2-001DD8B72B62,OV-NW-Connect Aalst 2019,2019-01-17 19:30:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
0A7B8EE9-C36B-E111-B43A-00505680000A,DB733356-A23D-ED11-9DB0-6045BD8952CE,OV-NW-Nieuwjaarsreceptie Aalst 2023,2023-01-26 18:30:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
01B45481-0877-E911-80FE-001DD8B72B62,5FC0DA29-E411-EA11-8108-001DD8B72B61,OV-JO-Breakfastclub Maart 2020,2020-03-04 08:00:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
01B45481-0877-E911-80FE-001DD8B72B62,CF7B3F01-51C4-EB11-8123-001DD8B72B62,OV-NW Zomerfeest 07 juli 2021,2021-07-07 17:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
...,...,...,...,...,...,...,...,...
1A096A0A-8072-EB11-8119-001DD8B72B61,342AA187-6116-EE11-8F6D-6045BD895A72,OV-IO-Techtrip 2024-Infosessie,2023-09-19 19:00:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
3A58CA88-5FFF-EB11-8125-001DD8B72B62,AA009971-2558-EC11-8F8F-000D3AAB8877,OV-NW-Voka Visit-Danilith Delmulle - Vlar - Wo...,2022-04-28 18:00:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
3A58CA88-5FFF-EB11-8125-001DD8B72B62,CD9F2FDB-3FB8-ED11-83FF-6045BD895D85,OV-NW- Voka Visit Graanstokerij Braeckman,2023-04-20 18:00:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5


In [27]:
#only show campagnes that will start in the future
similar_campagnes = similar_campagnes[similar_campagnes['Startdatum'] > '2023-11-16']
similar_campagnes

,CampagneId,CampagneNaam,Startdatum,Ondernemingsaard,Ondernemingstype,PrimaireActiviteit,Functie,rating
ContactPersoonId,,,,,,,,
01B45481-0877-E911-80FE-001DD8B72B62,D9834556-240C-EE11-8F6E-6045BD8956C9,OV-JO Community Event Starten NJ 2023,2023-11-29 16:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
16DEB180-171A-ED11-B83E-000D3AAB2014,5DE03B8C-5237-EE11-BDF4-000D3ABBC80D,OV-Internationale tewerkstelling 2023,2023-11-21 09:00:00,Diensten,Bedrijf,Consultancy,Bedrijfsleider,5
7CBB14DB-4165-EB11-811B-001DD8B72B62,08563B4A-6871-ED11-9561-6045BD8952CE,OV-Plato Academy 2023 - Workshop 4,2023-12-04 17:00:00,Diensten,Bedrijf,Consultancy,Contact Lidmaatschap,5
